In [1]:
import sys
sys.path.append('../src')
import preprocessing
import data_augmentation_1D as data_aug
import tensorflow as tf
from keras.layers import Conv1D, BatchNormalization,MaxPooling1D,UpSampling1D,Input
import keras 
from keras.optimizers import Adam
import matplotlib.pyplot as plt 
import numpy as np
import polars as pl
from keras.utils import to_categorical


2024-03-07 12:44:47.821346: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 12:44:47.821410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 12:44:47.959448: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 12:44:48.237008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 12:44:50.800917: W tensorflow/compiler/tf2

In [2]:
# Définir le chemin racine du dataset
dataset_root = "/mnt/c/Users/onerf/Documents/UMons/Master2/RHEA/src/db/"

# Charger les profils depuis les dossiers spécifiés dans le dataset
train_data  = preprocessing.sleepers_dataset_from_directory(dataset_root + "sleepers_db/")
test_data = preprocessing.sleepers_dataset_from_directory(dataset_root + "test/")

# Créer un dictionnaire pour mapper les identifiants uniques des cibles en entiers
id2int = {v: ix for ix, v in enumerate(np.unique(train_data.target))}
int2id = {ix: v for ix, v in enumerate(np.unique(train_data.target))}

# Convertir les cibles en valeurs entières pour faciliter leur manipulation par des algorithmes d'apprentissage automatique
train_data.target = [id2int[val] for val in train_data.target]
test_data.target = [id2int[val] for val in test_data.target]

train_data.target = to_categorical(train_data.target)
test_data.target = to_categorical(test_data.target)

x, y = train_data.data, train_data.target
# Redimensionnement des données pour ajouter une dimension supplémentaire,
x = x.reshape(x.shape[0], x.shape[1], 1)
print(f'x shape : {x.shape}')

x_test, y_test = test_data.data, test_data.target
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

print(f'x_test shape : {x_test.shape}')


2024-03-07 12:45:16.522699: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 12:45:16.926441: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 12:45:16.926538: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 12:45:16.930979: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-07 12:45:16.931081: I external/local_xla/xla/stream_executor

x shape : (2220, 437, 1)
x_test shape : (400, 437, 1)


In [3]:
x, y = data_aug.augment(x, y)

In [4]:
def encoder(input_size):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv1D(32, 3, activation='relu', padding='same')(input_size) 
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv1D(32, 3, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling1D(pool_size=2)(conv1) 
    conv2 = Conv1D(64, 3, activation='relu', padding='same')(pool1) 
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv1D(64, 3, activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling1D(pool_size=2)(conv2) 
    conv3 = Conv1D(128, 3, activation='relu', padding='same')(pool2) 
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv1D(128, 3, activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv1D(256, 3, activation='relu', padding='same')(conv3) 
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv1D(256, 3, activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    return conv4

def decoder(conv4):    
    #decoder
    conv5 = Conv1D(128, 3, activation='relu', padding='same')(conv4) 
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv1D(128, 3, activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv6 = Conv1D(64, 3, activation='relu', padding='same')(conv5) 
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv1D(64, 3, activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling1D(2)(conv6) 
    conv7 = Conv1D(32, 3, activation='relu', padding='same')(up1) 
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv1D(32, 3, activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling1D(2)(conv7) 
    decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(up2) 
    return decoded

In [5]:
batch_size = 128
epochs = 100
input_size = Input(shape = (437,1))
num_classes = 3

In [6]:
autoencoder = keras.Model(input_size, decoder(encoder(input_size)))
autoencoder.compile(loss='mean_squared_error', optimizer = Adam())

In [7]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 437, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 437, 32)           128       
                                                                 
 batch_normalization (Batch  (None, 437, 32)           128       
 Normalization)                                                  
                                                                 
 conv1d_1 (Conv1D)           (None, 437, 32)           3104      
                                                                 
 batch_normalization_1 (Bat  (None, 437, 32)           128       
 chNormalization)                                                
                                                                 
 max_pooling1d (MaxPooling1  (None, 218, 32)           0     

In [8]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
autoencoder_train = autoencoder.fit(x, x, batch_size=batch_size,epochs=epochs,verbose=1,validation_split=0.2,shuffle=True)


Epoch 1/100


ValueError: in user code:

    File "/home/faruk/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/faruk/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/faruk/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/faruk/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/faruk/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/home/faruk/anaconda3/envs/py310/lib/python3.10/si

In [ ]:
# Assuming your dataset is named 'data'
data_shape = x.shape

# Get the number of samples
num_samples = data_shape[0]

# Initialize a list to store the indices of samples with a different length
indices_with_different_length = []

# Iterate through the samples
for i in range(num_samples):
    sample_length = data_shape[1]  # Get the length of the current sample
    if sample_length != 437:
        indices_with_different_length.append(i)  # Store the index of the sample if its length is different

# Check if any samples have a different length
if len(indices_with_different_length) > 0:
    print("Samples with a different length found at indices:", indices_with_different_length)
else:
    print("All samples have the same length.")